In [21]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

In [16]:
origin=read_excel('행정동병합테스트22(반포본동).xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,612,2154,437,337.55,4,0.352898,0,32,0,3
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,488,1808,267,60.79,0,0.063554,0,27,0,12
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,108,626,110,956.51,3,1.000000,0,16,0,5
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,385,1814,311,0.00,3,0.000000,10,34,0,5
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,1033,3495,491,1.14,3,0.001192,10,54,0,5


In [17]:
origin.isna().sum()
origin1=origin.fillna(0)
scaler=MinMaxScaler()
origin1.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수'],
      dtype='object')

In [18]:
df = origin1.copy()
df['면적minmax']=scaler.fit_transform(df[['면적 (k㎡)']])
df['지하철minmax']=scaler.fit_transform(df[['지하철역_수']])
df['주차장minmax']=scaler.fit_transform(df[['개소 (공영+민영)']])
# df['주차장면수minmax']=scaler.fit_transform(df[['면수 (공영+민영)']])
df['버스정류장minmax']=scaler.fit_transform(df[['행정동별버스정류장수']])
df['공공자전거대여소수minmax']=scaler.fit_transform(df[['공공자전거대여소수']])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,면적minmax,지하철minmax,주차장minmax,버스정류장minmax,공공자전거대여소수minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.352898,0,32,0,3,0.187952,0.000000,0.209007,0.294118,0.065217
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.063554,0,27,0,12,0.080321,0.285714,0.111957,0.245098,0.260870
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,1.000000,0,16,0,5,0.101205,0.000000,0.105931,0.137255,0.108696
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.000000,10,34,0,5,0.163855,0.000000,0.186172,0.313725,0.108696
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.001192,10,54,0,5,0.693976,0.000000,0.509990,0.509804,0.108696


In [19]:
df['접근성지수'] = (df['지하철minmax'] + df['주차장minmax'] + df['버스정류장minmax'] + df['공공자전거대여소수minmax'])/ df['면적minmax']
# df['접근성지수'] = (df['지하철minmax'] + df['주차장면수minmax'] + df['버스정류장minmax'] + df['공공자전거대여소수minmax'])/ df['면적minmax']
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,면적minmax,지하철minmax,주차장minmax,버스정류장minmax,공공자전거대여소수minmax,접근성지수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0,32,0,3,0.187952,0.000000,0.209007,0.294118,0.065217,3.023873
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,27,0,12,0.080321,0.285714,0.111957,0.245098,0.260870,11.250303
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,0,16,0,5,0.101205,0.000000,0.105931,0.137255,0.108696,3.476923
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,10,34,0,5,0.163855,0.000000,0.186172,0.313725,0.108696,3.714208
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,10,54,0,5,0.693976,0.000000,0.509990,0.509804,0.108696,1.626123


In [20]:
df['접근성지수'].value_counts().sort_index()

접근성지수
0.503476     1
1.000131     1
1.173468     1
1.382370     1
1.513658     1
            ..
49.293889    1
57.638075    1
74.617210    1
85.609850    1
inf          2
Name: count, Length: 424, dtype: int64

In [22]:
df['접근성지수'].replace([np.inf, -np.inf], 0, inplace=True)
df

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,면적minmax,지하철minmax,주차장minmax,버스정류장minmax,공공자전거대여소수minmax,접근성지수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0,32,0,3,0.187952,0.000000,0.209007,0.294118,0.065217,3.023873
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,27,0,12,0.080321,0.285714,0.111957,0.245098,0.260870,11.250303
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,0,16,0,5,0.101205,0.000000,0.105931,0.137255,0.108696,3.476923
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,10,34,0,5,0.163855,0.000000,0.186172,0.313725,0.108696,3.714208
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,10,54,0,5,0.693976,0.000000,0.509990,0.509804,0.108696,1.626123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0,3,62,...,4,18,1,5,0.035341,0.428571,0.402474,0.156863,0.108696,31.028899
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0,1,80,...,9,15,1,6,0.038554,0.142857,0.390739,0.127451,0.130435,20.529061
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0,1,103,...,0,36,2,7,0.110843,0.142857,0.547098,0.333333,0.152174,10.604715
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0,1,1,...,7,9,0,4,0.055422,0.142857,0.002220,0.068627,0.086957,5.424974


In [23]:
df['접근성지수'].isna().sum()

0

In [24]:
df['접근성지수'].unique()

array([ 3.02387267, 11.25030252,  3.47692349,  3.71420754,  1.62612283,
        9.00717359, 35.72478926, 17.5181303 , 13.46679212, 30.36621005,
       12.30543921, 11.14399634, 49.2938888 , 85.60985003,  0.        ,
       28.10143569, 12.45877347, 21.60260407, 16.06711605,  7.75979911,
        5.04646043, 18.59152766, 21.02734214, 26.42351023, 20.29289242,
       26.71081681, 11.51329012, 14.32750839, 57.63807538, 25.0992387 ,
       13.30050859,  9.15785617,  4.23046703, 11.16326516, 11.33770438,
       15.90171078, 12.23814583, 33.29621548, 74.61721026,  9.2326314 ,
        1.64325089,  1.60126252, 18.64082282,  7.76644558,  5.98805362,
        2.93893389,  6.64333074, 18.29973463, 12.94662737, 11.77333553,
       11.42983308, 30.97847001, 27.92826767, 19.4227055 , 23.1269373 ,
       21.29421357, 10.72152941,  5.57456209,  5.85864704,  8.45317054,
        7.03869459, 10.90011096,  9.3380832 ,  4.22656309, 12.9057987 ,
       20.0276892 , 30.43089624, 25.23686604, 26.76352531,  5.62

In [25]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '면적minmax', '지하철minmax', '주차장minmax', '버스정류장minmax',
       '공공자전거대여소수minmax', '접근성지수'],
      dtype='object')

In [26]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','면적 (k㎡)','개소 (공영+민영)','지하철역_수','행정동별버스정류장수','공공자전거대여소수','면적minmax','지하철minmax','주차장minmax','버스정류장minmax','공공자전거대여소수minmax','접근성지수'])
# df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','면적 (k㎡)','면수 (공영+민영)','지하철역_수','행정동별버스정류장수','공공자전거대여소수','면적minmax','지하철minmax','주차장면수minmax','버스정류장minmax','공공자전거대여소수minmax','접근성지수'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),개소 (공영+민영),지하철역_수,행정동별버스정류장수,공공자전거대여소수,면적minmax,지하철minmax,주차장minmax,버스정류장minmax,공공자전거대여소수minmax,접근성지수
0,11110,종로구,1111051500,청운효자동,2.57,668,0,32,3,0.187952,0.000000,0.209007,0.294118,0.065217,3.023873
1,11110,종로구,1111053000,사직동,1.23,362,2,27,12,0.080321,0.285714,0.111957,0.245098,0.260870,11.250303
2,11110,종로구,1111054000,삼청동,1.49,343,0,16,5,0.101205,0.000000,0.105931,0.137255,0.108696,3.476923
3,11110,종로구,1111055000,부암동,2.27,596,0,34,5,0.163855,0.000000,0.186172,0.313725,0.108696,3.714208
4,11110,종로구,1111056000,평창동,8.87,1617,0,54,5,0.693976,0.000000,0.509990,0.509804,0.108696,1.626123


In [27]:
df1['접근성지수minmax']=scaler.fit_transform(df1[['접근성지수']])
df1

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),개소 (공영+민영),지하철역_수,행정동별버스정류장수,공공자전거대여소수,면적minmax,지하철minmax,주차장minmax,버스정류장minmax,공공자전거대여소수minmax,접근성지수,접근성지수minmax
0,11110,종로구,1111051500,청운효자동,2.57,668,0,32,3,0.187952,0.000000,0.209007,0.294118,0.065217,3.023873,0.035322
1,11110,종로구,1111053000,사직동,1.23,362,2,27,12,0.080321,0.285714,0.111957,0.245098,0.260870,11.250303,0.131414
2,11110,종로구,1111054000,삼청동,1.49,343,0,16,5,0.101205,0.000000,0.105931,0.137255,0.108696,3.476923,0.040614
3,11110,종로구,1111055000,부암동,2.27,596,0,34,5,0.163855,0.000000,0.186172,0.313725,0.108696,3.714208,0.043385
4,11110,종로구,1111056000,평창동,8.87,1617,0,54,5,0.693976,0.000000,0.509990,0.509804,0.108696,1.626123,0.018995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,11740,강동구,1174065000,성내제2동,0.67,1278,3,18,5,0.035341,0.428571,0.402474,0.156863,0.108696,31.028899,0.362445
422,11740,강동구,1174066000,성내제3동,0.71,1241,1,15,6,0.038554,0.142857,0.390739,0.127451,0.130435,20.529061,0.239798
423,11740,강동구,1174068500,길동,1.61,1734,1,36,7,0.110843,0.142857,0.547098,0.333333,0.152174,10.604715,0.123873
424,11740,강동구,1174069000,둔촌제1동,0.92,16,1,9,4,0.055422,0.142857,0.002220,0.068627,0.086957,5.424974,0.063369


In [28]:
df1.to_excel('접근성지수1.xlsx')